# Player Session Insights
* Use jupyter notebook and spark to support your answers.
* How many sessions are in the dataset?
* How many sessions are completed per country?
* How many sessions are completed per player?
* What is the country with more sessions started during 2018?
* Plot the player sessions completed per country

## Gather the dataset
https://cdn.unityads.unity3d.com/assignments/assignment_data.jsonl.bz2 (30-40 MB)

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [7]:
# I downloaded the json file
df = spark.read.json('assignment_data.jsonl.bz2')

In [11]:
# examine basic structure
df.is_cached()
df.printSchema()
print(df.head(2))

root
 |-- country: string (nullable = true)
 |-- event: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- ts: string (nullable = true)

[Row(country='PK', event='start', player_id='d6313e1fb7d247a6a034e2aadc30ab3f', session_id='674606b1-2270-4285-928f-eef4a6b90a60', ts='2016-11-22T20:40:50'), Row(country=None, event='end', player_id='20ac16ebb30a477087c3c7501b1fce73', session_id='16ca9d01-d240-4527-9f8f-00ef6cddb1d4', ts='2016-11-18T06:24:50')]


## Getting insight
* How many sessions are in the dataset? 1001169 sessions

In [12]:
df.count()

1001169